# Programming Assignment 4.3

In [271]:
import numpy as np
import pandas 
from scipy.spatial.distance import euclidean 
from sklearn.metrics import roc_auc_score as roc


Загрузите данные из файла data-logistic.csv. Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.

In [4]:
data = pandas.read_csv('data_logistic.csv', header = None)
print data.head()

,0,1,2
0,-1,-0.663827,-0.138526
1,1,1.994596,2.468025
2,-1,-1.247395,0.749425
3,1,2.309374,1.899836
4,1,0.849143,2.407750


Убедитесь, что выше выписаны правильные формулы для градиентного спуска. Обратите внимание, что мы используем полноценный градиентный спуск, а не его стохастический вариант!

Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) логистической регрессии. Используйте длину шага k=0.1. В качестве начального приближения используйте вектор (0, 0). Запустите градиентный спуск и доведите до сходимости (евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5). Рекомендуется ограничить сверху число итераций десятью тысячами.


In [275]:
def loglik(y, x1, x2, weights, x1_der = False, C = 0, k =0.1):
    if x1_der:
        lnLik_vector = []
        for i in xrange(len(y)):
            lnLik_i = y[i]*x1[i]*(1 - 1.0/(1+math.exp(-y[i]*(weights[0]*x1[i]+weights[1]*x2[i])))) - k*C*weights[0]
            lnLik_vector.append(lnLik_i)
    else:
        lnLik_vector = []
        for i in xrange(len(y)):
            lnLik_i = y[i]*x2[i]*(1 - 1.0/(1+math.exp(-y[i]*(weights[0]*x1[i]+weights[1]*x2[i])))) - k*C*weights[1]
            lnLik_vector.append(lnLik_i)
    return(sum(lnLik_vector))

def gradient_descent(y, x1, x2, k = 0.1, start = [0.0, 0.0], L2 = False, max_iter = 10000):
    if L2 == False :
        current_iter = 0 
        n_obs = len(y)
        w1 = start[0]
        w2 = start[1]
        while current_iter < max_iter:
            weights_previous = np.array([w1, w2])
            w1 = w1 + k*(1.0/n_obs)*loglik(y, x1, x2, weights = weights_previous, x1_der = True)
            w2 = w2 + k*(1.0/n_obs)*loglik(y, x1, x2, weights = weights_previous, x1_der = False)
            current_iter += 1 
            weights_current = np.array([w1, w2])
            if euclid(weights_current, weights_previous) < pow(10, -5):
                print 'Euclid:', euclid(weights_current, weights_previous)
                break 
    if L2:
        current_iter = 0 
        n_obs = len(y)
        w1 = start[0]
        w2 = start[1]
        while current_iter < max_iter:
            weights_previous = np.array([w1, w2])
            w1 = w1 + k*(1.0/n_obs)*loglik(y, x1, x2, weights = weights_previous, x1_der = True, C = 10) 
            w2 = w2 + k*(1.0/n_obs)*loglik(y, x1, x2, weights = weights_previous, x1_der = False, C = 10)
            current_iter += 1 
            weights_current = np.array([w1, w2])
            if euclid(weights_current, weights_previous) < pow(10, -5):
                #print 'Euclid:', euclid(weights_current, weights_previous)
                break 
    
    return(weights_current)

    
            

In [270]:
%time print gradient_descent(data[0], data[1], data[2], k = 0.1, start = [0.0, 0.0], L2 = False, max_iter = 1000)
%time print gradient_descent(data[0], data[1], data[2], k = 0.1, start = [0.0, 0.0], L2 = True, max_iter = 1000, C = 10)

Euclid: 9.80234978163e-06
(array([ 0.28781162,  0.0919833 ]), 244)
CPU times: user 9.69 s, sys: 141 ms, total: 9.83 s
Wall time: 9.87 s
Euclid: 8.98732633015e-06
(array([ 0.12328347,  0.08684528]), 45)
CPU times: user 1.9 s, sys: 71 ms, total: 1.97 s
Wall time: 1.97 s


Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании? Эти величины будут ответом на задание. В качестве ответа приведите два числа через пробел. Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом. Для этого воспользуйтесь сигмоидной функцией: a(x) = 1 / (1 + exp(-w1 x1 - w2 x2)).

In [284]:
predval_noreg = []
x1 = data[1]
x2 = data[2]
weights_noreg = gradient_descent(data[0], data[1], data[2], k = 0.1, start = [0.0, 0.0], L2 = False, max_iter = 1000)

for i in xrange(len(y)):
    predval_noreg.append(1/(1+(math.exp(-weights_noreg[0]*x1[i] - weights_noreg[1]*x2[i]))))

weights_reg = gradient_descent(data[0], data[1], data[2], k = 0.1, start = [0.0, 0.0], L2 = True, max_iter = 1000)
predval_reg = []
for i in xrange(len(y)):
    predval_reg.append(1/(1+(math.exp(-weights_reg[0]*x1[i] - weights_reg[1]*x2[i]))))



Euclid: 9.80234978163e-06


In [285]:
roc1 = roc(y_true = data[0], y_score = predval_noreg)
roc2 = roc(y_true = data[0], y_score = predval_reg)
print roc1, roc2

0.926857142857 0.937047619048


In [244]:
def euclid(x1, x2):
    values_to_sum = []
    for i in xrange(len(x1)):
        values_to_sum.append(pow((x1[i]-x2[i]),2))
    total = np.sqrt(sum(values_to_sum))
    return(total)

def partial_derivative_lnLik(y, x1, x2, weights, x1_der = False):
    if x1_der:
        X = pandas.DataFrame([x1, x2]).as_matrix() # shape 2 * n
        w_X = weights.dot(X)
        inner_part_denominator_before_exp = - np.transpose(y).dot(w_X)
        inner_part_denominator = 1 + np.exp(inner_part_denominator_before_exp)
        part1 = 1 - (1.0/inner_part_denominator)
        total = np.transpose(y).dot(x1.as_matrix())*part1
    else: 
        X = pandas.DataFrame([x1, x2]).as_matrix() # shape 2 * n
        w_X = weights.dot(X)
        inner_part_denominator_before_exp = - np.transpose(y).dot(w_X)
        inner_part_denominator = 1 + np.exp(inner_part_denominator_before_exp)
        part1 = 1 - (1.0/inner_part_denominator)
        total = np.transpose(y).dot(x2.as_matrix())* part1
    return(total)

def gradient_descent(y, x1, x2, k = 0.1, start = [0.0, 0.0], L2 = False, max_iter = 10000):
    if L2 == False :
        current_iter = 0 
        n_obs = len(y)
        w1 = start[0]
        w2 = start[1]
        while current_iter < max_iter:
            weights_previous = np.array([w1, w2])
            w1 = w1 + k*(1.0/n_obs)*partial_derivative_lnLik(y, x1, x2, weights = weights_previous, x1_der = True)
            w2 = w2 + k*(1.0/n_obs)*partial_derivative_lnLik(y, x1, x2, weights = weights_previous, x1_der = False)
            current_iter += 1 
            weights_current = np.array([w1, w2])
            if euclid(weights_current, weights_previous) < pow(10, -9):
                print 'Euclid:', euclid(weights_current, weights_previous)
                break 
    if L2:
        current_iter = 0 
        n_obs = len(y)
        w1 = start[0]
        w2 = start[1]
        while current_iter < max_iter:
            weights_previous = np.array([w1, w2])
            w1 = w1 + k*(1.0/n_obs)*partial_derivative_lnLik(y, x1, x2, weights = weights_previous, x1_der = True)
            w2 = w2 + k*(1.0/n_obs)*partial_derivative_lnLik(y, x1, x2, weights = weights_previous, x1_der = False)
            current_iter += 1 
            weights_current = np.array([w1, w2])
            if euclid(weights_current, weights_previous) < pow(10, -9):
                print 'Euclid:', euclid(weights_current, weights_previous)
                break 
    
    return(weights_current, current_iter)


In [245]:
%time print gradient_descent(data[0], data[1], data[2], k = 0.1, start = [0.0, 0.0], L2 = False, max_iter = 1000)


(array([ 0.04574334,  0.04155541]), 10000)
CPU times: user 1min 31s, sys: 292 ms, total: 1min 31s
Wall time: 1min 32s


In [257]:
def loglik(y, x1, x2, weights, x1_der = False):
    if x1_der:
        lnLik_vector = []
        for i in xrange(len(y)):
            lnLik_i = y[i]*x1[i]*(1 - 1.0/(1+math.exp(-y[i]*(weights[0]*x1[i]+weights[1]*x2[i])))) 
            lnLik_vector.append(lnLik_i)
    else:
        lnLik_vector = []
        for i in xrange(len(y)):
            lnLik_i = y[i]*x2[i]*(1 - 1.0/(1+math.exp(-y[i]*(weights[0]*x1[i]+weights[1]*x2[i])))) 
            lnLik_vector.append(lnLik_i)
    return(sum(lnLik_vector))

print loglik(ytr, x1tr, x2tr, weights, x1_der = True)

73.2490961174


In [154]:


x1 = np.array([1, 2, 3])
x2 = np.array([1, 2, 3])

print euclid(x1, x2)


0.0


In [14]:
y = [1,10]
x = [1,10]


TypeError: can't multiply sequence by non-int of type 'list'

In [185]:
ytr = pandas.Series([1,2,3])
x1tr = pandas.Series([1, 5, 3])
x2tr = pandas.Series([4,8,2])
weights = np.array([2,3])
partial_derivative_lnLik(y = ytr, x1 = x1tr, x2 = x2tr, weights = weights, x1_der = True)


0.0

In [223]:
x1tr = data[1]
x2tr = data[2]
ytr = data[0]
weights = np.array([0,0])
X = pandas.DataFrame([x1tr, x2tr]).as_matrix() # shape 2 * n
w_X = weights.dot(X)
#print np.transpose(ytr)
inner_part_denominator_before_exp = - np.transpose(ytr).dot(w_X)
print inner_part_denominator_before_exp
print np.exp(inner_part_denominator_before_exp)
inner_part_denominator = 1.0 + np.exp(inner_part_denominator_before_exp)
print inner_part_denominator
part1 = 1 - (1.0/inner_part_denominator)
print part1
total = np.transpose(y).dot(x1tr.as_matrix())*part1
print total

-0.0
1.0
2.0
0.5
73.2490961174
